In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from stop_words import get_stop_words
from sklearn.metrics import classification_report
import csv
from ipywidgets import widgets
from IPython.display import display
from playsound import playsound

%config InlineBackend.figure_format = 'retina'

In [2]:
# Change working directory to get the file
try:
    os.chdir('/Users/pkg/Documents/OneDrive - Elektroforeningen (EFO)/Python/Intermediate DS')
except:
    os.chdir('/Users/paal/OneDrive - Elektroforeningen (EFO)/Python/Intermediate DS')

# Open pickled file from the data wrangling section and set working directory
with open('data_wrangling.pickle', 'rb') as handle:
    df = pickle.load(handle)
    
try:
    os.chdir('/Users/pkg/Documents/OneDrive - Elektroforeningen (EFO)/Python/Intermediate DS/Intermediate Data Science with Python/Python_Capstone')
except:
    os.chdir('/Users/paal/OneDrive - Elektroforeningen (EFO)/Python/Intermediate DS/Intermediate Data Science with Python/Python_Capstone')

In [3]:
# Sort by df by ETIM_classes for the clf.predict_proba-method later
df = df.sort_values(['ETIM_class', 'ENG'], ascending=True)

In [4]:
# Separating the labels from the rest of the data set
labels = df['ETIM_class']
ex_var = df[['ENG', 'Technical_description']]

# Display some attributes of the dataset
print("labels'  shape:", labels.shape)
print("ex_var's shape:", ex_var.shape)
print("")
print("labels is of type", type(labels))
print("ex_var is of type", type(ex_var))
print("")
print("first label after sorting:", labels.iloc[0])
print("first ENG after sorting:", ex_var.iloc[0,0])
print("first technical description after sorting:", ex_var.iloc[0,1])

labels'  shape: (202216,)
ex_var's shape: (202216, 2)

labels is of type <class 'pandas.core.series.Series'>
ex_var is of type <class 'pandas.core.frame.DataFrame'>

first label after sorting: EC000001
first ENG after sorting: 11
first technical description after sorting: Direkte tilkoblingsklemmer Al/Cu 70/300mm2,sett à 4 stk. L1/L2/L3/PEN


In [5]:
# Importing stop words that will be ignored
stop_words = get_stop_words('norwegian')

# Creating a Pandas Series of the technical descriptions
text = df['Technical_description']

# Creating the corpus
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1,2), min_df=2)

# Build the vocabulary
vectorizer.fit(text)

# Convert text to a bag of words, returns a Compressed Sparse Row matrix
# This is suitable for a matrix that is primarily made up of zeroes.
x = vectorizer.transform(text)

In [6]:
# Setting up X and y
X = x
y = labels

# Create the test and training sets
xtrain, xtest, ytrain, ytest = train_test_split(X, y, random_state=99)

In [7]:
# Train the classifier over the training set, and test on the test set
clf = MultinomialNB(alpha=0.0000000001).fit(xtrain, ytrain)
#NB_train_accuracy = clf.score(xtrain, ytrain)
#NB_test_accuracy = clf.score(xtest, ytest)

# Accuracy scores for both the training and test sets
#print("Training accuracy:", round(NB_train_accuracy, 2))
#print("Testing accuracy", round(NB_test_accuracy, 2))

In [8]:
# Creating the classification report
#y_pred_train = clf.predict(xtrain)
#y_pred_test = clf.predict(xtest)

#print(classification_report(y_pred_train, ytrain))
#print(classification_report(y_pred_test, ytest))

In [9]:
# Dictionary to connect the ETIM-class-codes and the descriptions (in Norwegian)
ETIM_dict = pd.read_csv('ETIM7.csv', header=None, sep=';', index_col=0).to_dict()[1]
#ETIM_dict

In [10]:
# Query for single technical descriptionsvestre v
inputText = widgets.Text(value='Fyll inn teknisk beskrivelse her')

# Create query
def predict_ETIM(sender):

    # Creating technical description to be evaluated by the model
    text = inputText.value
    example = clf.predict_proba(vectorizer.transform([text])).flatten()

    # Display the predicted class
    test = clf.predict(vectorizer.transform([text]))
    #test[0]

    # Create a dictionary for  the classes and probabilities
    example_dict = dict(zip(sorted(ytrain.unique()), example))
    #example_dict

    # Display the top 3 predictions, with classes and probabilities
    top3 = {k: example_dict[k] for k in sorted(example_dict, key=example_dict.__getitem__, reverse=True)[:3]}
    #top3[test[0]]

    # Shorten to integer percentage
    #"%.0f" % (top3[test[0]] * 100)
    top3s = sorted(example_dict, key=example_dict.__getitem__, reverse=True)[:3]

    print('Tekst:', text)
    print('')
    
    for i in range(len(top3s)):

        if top3[top3s[i]] < 0.1:
            break
        else:
            print('Forslag', int(i+1), ':')
            print('Forventet ETIM-klasse:', top3s[i], ETIM_dict[top3s[i]])
            print('Forventet treffsikkerhet:', "%.0f" % (top3[top3s[i]] * 100), '%')
            print('')
    
inputText.on_submit(predict_ETIM)
inputText

Text(value='Fyll inn teknisk beskrivelse her')

In [11]:
## Importing Excel-file with technical descriptions that need to be classified

# Assign spreadsheet filename to 'file'
file = 'import_mw.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file).parse('Sheet1')

# See first technical description and length of file
print(xl.iloc[0,:])
len(xl)

elnr                                                             1426550
teknisk beskrivelse    FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-1...
Name: 0, dtype: object


139

In [12]:
# Create an empty list to store the analysis results
rows_list = []

# Predict ETIM-classes for a number of technical descriptions (for use in analyze_xl, not standalone use)
# For standalone use, see predict_ETIM()
def predict_ETIM_raw(elnr, text):
    
    # Creating technical description to be evaluated by the model
    example = clf.predict_proba(vectorizer.transform([text])).flatten()

    # Display the predicted class
    single_prod = clf.predict(vectorizer.transform([text]))

    # Create a dictionary for the classes and probabilities
    prob_dict = dict(zip(sorted(ytrain.unique()), example))

    # Top 3 predictions, with classes and probabilities
    top3 = {k: prob_dict[k] for k in sorted(prob_dict, key=prob_dict.__getitem__, reverse=True)[:3]}

    # Top 3 predictions in sorted order
    top3s = sorted(prob_dict, key=prob_dict.__getitem__, reverse=True)[:3]

    for i in range(len(top3)):
        
        single_prod = {elnr: [k for k in [text, top3s[i], ETIM_dict[top3s[i]], "%.0f" % (top3[top3s[i]] * 100)]]}
        
        if top3[top3s[i]] < 0.1:
            break
        else:
            print(single_prod)
            temp_dict = {}
            temp_dict.update(single_prod)
            rows_list.append(temp_dict)
           

In [13]:
# Run the analysis on the data from the Excel-file
def analyze_xl(file):
    
    # Find text from Excel-file to analyze
    for j in range(len(xl)):
        predict_ETIM_raw(xl.iloc[j,0], xl.iloc[j,1])

In [14]:
# Run the analysis (prints results and writes to rows_list)
analyze_xl(xl)

{1426550: ['FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-1000LUX, FORSINKET PÅ 1-15 SEK, FORSINKET AV 10-30SEK, FORBRUK 0,8W, -25 TIL 50° C, TILKOBLING 2,5MM²', 'EC001744', 'Spotlight', '100']}
{1426551: ['FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A, JUSTERBAR 2-1000LUX, FORSINKET PÅ 1-15 SEK, FORSINKET AV 10-30SEK, FORBRUK 0,56W, -25 TIL 50° C, TILKOBLING 2,5MM²', 'EC000133', 'Bevegelsessensor komplett', '87']}
{1426551: ['FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A, JUSTERBAR 2-1000LUX, FORSINKET PÅ 1-15 SEK, FORSINKET AV 10-30SEK, FORBRUK 0,56W, -25 TIL 50° C, TILKOBLING 2,5MM²', 'EC002892', 'Tak og veggarmatur', '13']}
{1426554: ['KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X VEKSELKONTAKTER A 16A, +/- 1 SEK PR 24T, 0,56W, -25 TIL 50° C, TILKOBLING 2,5MM², AUTO JUSTERING SOMMER/VINTERTID, DIN MONTERING.', 'EC002305', 'Digitalt koblingsur for sikringsskap', '100']}
{1426555: ['KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC, 2 X VEKSELKONTAKTER A 16A, +/- 1 SEK PR 24T, 0,56W, -25 TIL

{1611201: ['FORDELINGSKINNE 2P, 16 MODULER  FOR MT OG MDC, U TYPE', 'EC000003', 'Jordfeilbryter', '100']}
{1611202: ['FORDELINGSKINNE 2P, 18 MODULER, MT OG MDC, U TYPE  ', 'EC001286', 'Hjelpekontaktenhet for fordelingstavle', '100']}
{1611203: ['FORDELINGSKINNE 2P, 20 MODULER, MT OG MDC, U TYPE  ', 'EC002297', 'Jordfeilbryter', '61']}
{1611203: ['FORDELINGSKINNE 2P, 20 MODULER, MT OG MDC, U TYPE  ', 'EC001286', 'Hjelpekontaktenhet for fordelingstavle', '23']}
{1611203: ['FORDELINGSKINNE 2P, 20 MODULER, MT OG MDC, U TYPE  ', 'EC000003', 'Jordfeilbryter', '16']}
{1611204: ['FORDELINGSKINNE 2P, 21 MODULER, MT OG MDC, U TYPE  ', 'EC001286', 'Hjelpekontaktenhet for fordelingstavle', '96']}
{1611205: ['FORDELINGSKINNE 3P, 21 MODULER, MT OG MDC, U TYPE  ', 'EC002297', 'Jordfeilbryter', '90']}
{1611205: ['FORDELINGSKINNE 3P, 21 MODULER, MT OG MDC, U TYPE  ', 'EC000003', 'Jordfeilbryter', '10']}
{1611815: ['OVERSPENNINGSVERN, 2P, 20kA, Uc 385, KL.II/C, 2 MODULER FOR IT/TT NETT', 'EC000941', 'Ov

{4302166: ['MFZ61DX-UC, 1 NO, 10A/250V potensialfri kontakt. Styrespenning universal fra 8-230VAC/DC. Bryter og slutter i sinuskurvens 0 punkt. Funksjoner RV Forsinket ut, AV forsinket inn, TI klokke puls start ved innkopling, EW puls ved innkopling, AW puls ved frakopling, IA impulsstyrt forsinket inn. Tidsområde 0,5 sek til 1 time. Bruksområder Styring av lys, kontakter eller annet. Styres med en eller flere vanlige brytere, fotocelle, astrour, bevegelse sensor osv. Maks kapasitet kan benyttes når en timer eller klokkepuls er satt til ≥ 5 minutter. ≤ 2 sekunder 15 % av maks belastning, ≤ 2 min 30 % av maks belastning, ≤ 5 minutter 60 % av maks belastning', 'EC001669', 'Tidsrelé for DIN-skinne', '100']}
{4513411: ['Fjernkontroll, FMH4-wg, uten batteri, 4 kanaler. Hvit. 43x43x16mm (lxbxd)', 'EC001587', 'Hånd-/veggsender for bussystem', '100']}
{4513800: ['Trådløse bryter programmerbar, 4 kanaler, hvit, hel vippe eller kronevender, bredde ytter ramme 80x80, mål bryter 55x55, passer i RS

{4513976: ['Bevegelse, lux, temp og fuktighet sensor FBH66TFB-wg. 10-2000 lux, temperatur -20°C til + 60°C, fuktighet fra 0% to 100%. Frittstående montasje på vegg eller i stad veggboks rekkevidde 10m. Takmontert (h=3m) diameter 5m, (h=2,5m) diameter 4m. Glanset hvit. Yttermål 84x84x39mm', 'EC001582', 'Bevegelsessensor for bussystem', '100']}
{4513978: ['Trådløs mini smykke  bryter med 4 kanaler, batteriløs. FMH2S-wr', 'EC001097', 'Bryteraktuator for bussystem', '100']}
{4514178: ['Smart hus kontroll enhet for Eltakos Tap-radio, serie 61 og serie 14 aktauorer, brytere og sensorer. Kan monteres på vegg eller på DIN skinne med medfølgende braketter. Tilkobling for ekstern antenne. Processor: Intel X1021, minne: 1 GB DDR3 RAM + 4 GB eMMC, LAN: 1x 10/100Mbit ethernet, integrert EnOcean-868MHz trådløs modul(TCM310) ESP3, strømforyning 5V, plastikk hus glanset hvit. Mål: 165x70x35mm (LxBxD) Veight: 175 grams. Styres med appen GFA4.\n', 'EC000534', 'Styreenhet for lysstyring', '100']}
{451418

{4514287: ['Startkit universal dimmer med FT55-wg (hvit glanset) trådløs bryter og FUD61 universal dimme aktuator. 300W Ohmsk, induktiv og kapasitiv last, 100W 230V LED og 230V dimmebare sparepærer.', 'EC001094', 'Dimmeaktuator for bussystem', '100']}
{4514288: ['Startkit bryte aktuator med FT55-wg trådløs bryter og FSR61-230 bryte aktuator med 1 stk 1P 10A/250V potesialfri kontakt. Med eller uten forsinket ut, av/på eller impuls bryter.', 'EC001097', 'Bryteraktuator for bussystem', '100']}
{4514290: ['FRP61-230V, Trådløs forsterker (repeater)', 'EC000409', 'Booster / signalforsterker', '60']}
{4514290: ['FRP61-230V, Trådløs forsterker (repeater)', 'EC000698', 'Repeater for bussystem', '40']}
{4514454: ["Trådløs trykk og vri bryter, glanset hvit 84x84x25mm kan monteres i E-design rammer. Med batteri. Trykk og vri bryter som kan styre alle Eltako's universal dimme aktuatorer samt bryte\naktuatorene TF61L, TF100L og alle FSR61.Innlæring av trykk og vri bryteren; Fjerne front rammen og ra

{4514491: ['TF-4FT55: 4-kanals trådløs, batteri løs bryter. Glanset hvit, 80x80x15mm, inne ramme 55x55 passer i RS16 rammer. Lager egen energi når bryteren trykkes inn for å sende trådløse EnOcean\ntelegram. 4 kanal trådløse brytere med hel vippe kan sende to forskjellige telegrammer, og dermed kontrollere to kanaler: dvs. et på toppen av vippa og et nederst på vippa. Festeplaten kan skrues på en flat overflate eller limes til vegg, glass eller møbler ved hjelp av det medfølgende limeplaten. Bruk de forehåndsborede hullene på montering braketten til montering på skruen over en innfelt boks. TF100L-230 V: Trådløs plugg inn bryte aktuator med impuls kontakt. 1NO, 10 A/250 V AC. 100 x 55 x 45 mm. Hvit. Gløde, halogen pærer og LED m/driver inntil 1000W, Sparepærer og 230V LED inntil 200 W. Kun 0,8 W effektforbruk i stand-by. Passer stikk av Norske standard. Barne beskyttede uttak. Inntil 24 trådløse bryter (universal, retningsstyrt, sentralt av/på eller bevegelse sensor TF-BSB/TF-BSB55 og 

In [28]:
# Convert the dictionary to a dataframe
output_df = pd.DataFrame(rows_list).T
output_df

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,149,150,151,152,153
1426550,"[FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426551,NaN,"[FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A...","[FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426554,NaN,NaN,NaN,"[KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X V...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426555,NaN,NaN,NaN,NaN,"[KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426567,NaN,NaN,NaN,NaN,NaN,"[Bevegelse/tilstedeværelse detektor, 360°, Jus...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438860,NaN,NaN,NaN,NaN,NaN,NaN,[Input kontroller SUD12/1-10V. Kan settes opp ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""Trykk og vri universal dimmer DTD55. DTD55 k...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Trykk og vri universal dimmer DTD55L. DTD55L ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTI...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Squeeze the values to the left
def squeeze_nan(x):
    original_columns = x.index.tolist()

    squeezed = x.dropna()
    squeezed.index = [original_columns[n] for n in range(squeezed.count())]

    return squeezed.reindex(original_columns, fill_value=np.nan)

# Run the function and remove rightmost columns with only Nan
# Nr. of columns need to be correct according to output_df above
output_df = output_df.apply(squeeze_nan, axis=1)
output_df = output_df.dropna(axis=1, how='all')
output_df.columns = ['anbf1', 'anbf2', 'anbf3']
output_df

,anbf1,anbf2,anbf3
1426550,"[FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-...",NaN,NaN
1426551,"[FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A...","[FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A...",NaN
1426554,"[KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X V...",NaN,NaN
1426555,"[KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC,...",NaN,NaN
1426567,"[Bevegelse/tilstedeværelse detektor, 360°, Jus...",NaN,NaN
1438860,[Input kontroller SUD12/1-10V. Kan settes opp ...,NaN,NaN
1438863,"[""Trykk og vri universal dimmer DTD55. DTD55 k...",NaN,NaN
1438864,[Trykk og vri universal dimmer DTD55L. DTD55L ...,NaN,NaN
1438880,"[UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTI...",NaN,NaN
1438881,"[UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTI...",NaN,NaN


In [30]:
# Concatinate
output_df1 = pd.concat([output_df['anbf1'], output_df['anbf1'].apply(pd.Series).add_prefix('nr_')], axis=1)
output_df2 = pd.concat([output_df['anbf2'], output_df['anbf2'].apply(pd.Series).add_prefix('nr_')], axis=1)
output_df3 = pd.concat([output_df['anbf3'], output_df['anbf3'].apply(pd.Series).add_prefix('nr_')], axis=1)

In [31]:
# Concatinate files
output_df = pd.concat([output_df1, output_df2, output_df3], axis=1)
output_df.columns = output_df.columns.astype(str)
output_df

,anbf1,nr_0,nr_1,nr_2,nr_3,anbf2,nr_0,nr_1,nr_2,nr_3,anbf3,nr_0,nr_1,nr_2,nr_3
1426550,"[FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-...","FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-1...",EC001744,Spotlight,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426551,"[FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A...","FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A,...",EC000133,Bevegelsessensor komplett,87,"[FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A...","FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A,...",EC002892,Tak og veggarmatur,13,NaN,NaN,NaN,NaN,NaN
1426554,"[KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X V...","KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X VE...",EC002305,Digitalt koblingsur for sikringsskap,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426555,"[KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC,...","KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC, ...",EC002305,Digitalt koblingsur for sikringsskap,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426567,"[Bevegelse/tilstedeværelse detektor, 360°, Jus...","Bevegelse/tilstedeværelse detektor, 360°, Just...",EC000133,Bevegelsessensor komplett,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438860,[Input kontroller SUD12/1-10V. Kan settes opp ...,Input kontroller SUD12/1-10V. Kan settes opp m...,EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438863,"[""Trykk og vri universal dimmer DTD55. DTD55 k...","""Trykk og vri universal dimmer DTD55. DTD55 ko...",EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438864,[Trykk og vri universal dimmer DTD55L. DTD55L ...,Trykk og vri universal dimmer DTD55L. DTD55L t...,EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438880,"[UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTI...","UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTIV...",EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438881,"[UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTI...","UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTIV...",EC000025,Dimmer,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Delete unconcatinated columns
output_df = output_df.drop(['anbf1', 'anbf2', 'anbf3'], 1)

output_df

,nr_0,nr_1,nr_2,nr_3,nr_0,nr_1,nr_2,nr_3,nr_0,nr_1,nr_2,nr_3
1426550,"FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-1...",EC001744,Spotlight,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426551,"FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A,...",EC000133,Bevegelsessensor komplett,87,"FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A,...",EC002892,Tak og veggarmatur,13,NaN,NaN,NaN,NaN
1426554,"KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X VE...",EC002305,Digitalt koblingsur for sikringsskap,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426555,"KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC, ...",EC002305,Digitalt koblingsur for sikringsskap,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426567,"Bevegelse/tilstedeværelse detektor, 360°, Just...",EC000133,Bevegelsessensor komplett,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438860,Input kontroller SUD12/1-10V. Kan settes opp m...,EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438863,"""Trykk og vri universal dimmer DTD55. DTD55 ko...",EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438864,Trykk og vri universal dimmer DTD55L. DTD55L t...,EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438880,"UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTIV...",EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438881,"UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTIV...",EC000025,Dimmer,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Rename columns
output_df.columns = ['Tekst', 'ETIM-kode', 'Navn', 'Forventet sannsynlighet', 'Tekst', 'ETIM-kode', 'Navn', 'Forventet sannsynlighet', 'Tekst', 'ETIM-kode', 'Navn', 'Forventet sannsynlighet']
output_df

,Tekst,ETIM-kode,Navn,Forventet sannsynlighet,Tekst,ETIM-kode,Navn,Forventet sannsynlighet,Tekst,ETIM-kode,Navn,Forventet sannsynlighet
1426550,"FOTOCELLE, UTVENDIG, IP65, <16A, JUSTERBAR 2-1...",EC001744,Spotlight,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426551,"FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A,...",EC000133,Bevegelsessensor komplett,87,"FOTOCELLE, INNVENDIG MED UTVENDIG FØLER, <16A,...",EC002892,Tak og veggarmatur,13,NaN,NaN,NaN,NaN
1426554,"KOBLINGSUR MED 2 KANALER, 24-264VAC/DC, 2 X VE...",EC002305,Digitalt koblingsur for sikringsskap,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426555,"KOBLINGSUR ASTRO MED 2 KANALER, 24-264VAC/DC, ...",EC002305,Digitalt koblingsur for sikringsskap,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1426567,"Bevegelse/tilstedeværelse detektor, 360°, Just...",EC000133,Bevegelsessensor komplett,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438860,Input kontroller SUD12/1-10V. Kan settes opp m...,EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438863,"""Trykk og vri universal dimmer DTD55. DTD55 ko...",EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438864,Trykk og vri universal dimmer DTD55L. DTD55L t...,EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438880,"UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTIV...",EC000025,Dimmer,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438881,"UNIVERSAL DIMMER FOR OHMSK LAST 400W, INDUKTIV...",EC000025,Dimmer,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Export the file to an Excel-file
writer = pd.ExcelWriter('output_mw.xlsx', engine='xlsxwriter')
#worksheet.conditional_format('')
output_df.to_excel(writer, 'Sheet1')
writer.save()

# Finished!
playsound('fanfare_ff2.mp3')